In [3]:
pip install scikeras

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier
import io
from tensorflow.keras.utils import to_categorical
from google.colab import files

# Carregar o conjunto de dados
#drive.mount('/content/drive')
df = pd.read_csv('https://raw.githubusercontent.com/andvolp/Projeto-X/main/Projeto%201/winequality-white.csv', encoding='utf-8', sep=';')

dfs = np.array(df)

for i in range(len(dfs)):
    if dfs[i,11] > 5:
      dfs[i,11] = 2
    elif dfs[i, 11] == 5:
      dfs[i, 11] = 1
    else:
      dfs[i, 11] = 0

# Utilizando o keras
X = dfs[:, :-1]
y = dfs[:, 11]

y = to_categorical(y, num_classes=3)

scaler = StandardScaler()
X = scaler.fit_transform(X)

class Contador:
  def __init__(self):
    self.cont = 0

  def soma(self):
    self.cont = self.cont + 1
    print(self.cont)


# Criar um conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=3)

contador = Contador()

# Função para construir o modelo com dropout_rate, neurons_layer1 e neurons_layer2
def build_model(dropout_rate, neurons_layer1, neurons_layer2):
    model = Sequential()
    model.add(Dense(neurons_layer1, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons_layer2, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(3, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    contador.soma()
    return model

# Criar um modelo KerasClassifier para uso com o GridSearchCV
model = KerasClassifier(model=build_model, dropout_rate=[0.1], neurons_layer1=[256], neurons_layer2=[256], verbose=1)

# Definir os hiperparâmetros para busca em grade
param_grid = dict(dropout_rate=[0.1, 0.2, 0.3, 0.4, 0.5], neurons_layer1=[64, 128, 256, 512, 1024], neurons_layer2=[64, 128, 256, 512, 1024])

# Criar o objeto GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1)

# Realizar a busca em grade no conjunto de treinamento
grid_result = grid.fit(X_train, y_train, epochs=100, batch_size=64, verbose=1)

# Imprimir os resultados
print("Melhores hiperparâmetros encontrados: ", grid_result.best_params_)
print("Melhor acurácia encontrada: ", grid_result.best_score_)

# Avaliar o modelo com os melhores hiperparâmetros no conjunto de teste
best_model = grid_result.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = np.mean(y_test == np.round(y_pred))
print('Acuracia no conjunto de teste com melhores hiperparâmetros: ', accuracy)

log_stream = io.StringIO()
log_stream.write(f"Accuracy: {accuracy}, Best parameters: {best_model}\n")
log_stream.write(str(grid_result))
log_stream.write(str(grid_result.best_score_))
log_stream.write(str(grid_result.best_params_))

 #Salvar o arquivo de log em disco
with open('history.log', 'w') as f:
    f.write(log_stream.getvalue())

files.download('/content/history.log')



Streaming output truncated to the last 5000 lines.
35/35 [==============================] - 0s 4ms/step - loss: 0.3029 - accuracy: 0.7866
Epoch 46/100
35/35 [==============================] - 0s 5ms/step - loss: 0.3017 - accuracy: 0.7971
Epoch 47/100
35/35 [==============================] - 0s 4ms/step - loss: 0.2960 - accuracy: 0.8007
Epoch 48/100
35/35 [==============================] - 0s 5ms/step - loss: 0.2956 - accuracy: 0.7984
Epoch 49/100
35/35 [==============================] - 0s 4ms/step - loss: 0.2952 - accuracy: 0.7911
Epoch 50/100
35/35 [==============================] - 0s 4ms/step - loss: 0.2951 - accuracy: 0.7994
Epoch 51/100
35/35 [==============================] - 0s 4ms/step - loss: 0.2993 - accuracy: 0.7925
Epoch 52/100
35/35 [==============================] - 0s 5ms/step - loss: 0.2918 - accuracy: 0.8039
Epoch 53/100
35/35 [==============================] - 0s 5ms/step - loss: 0.2925 - accuracy: 0.7952
Epoch 54/100
35/35 [==============================] - 0s 4ms/s